<a href="https://colab.research.google.com/github/nerudxlf/directory_of_names/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files

files.upload()

In [12]:
def join_names(tuple_name: tuple) -> str:
  result_string: str = ""
  for name in tuple_name:
    result_string += str(name) + ";"
  return result_string


def just_join_names(tuple_names: tuple) -> str:
  result_str: str = ""
  for name in tuple_names:
    result_str += str(name)
  return result_str


def filter_names(data_list: list) -> list:
  return_list: list = []
  for item in data_list:
    item = item.replace("nan;", "")
    item = item.replace("0;", "")
    return_list.append(item.lower())
  return return_list


def get_from_ready_excel():
  """Формирует DF full data из 2х таблиц с именами"""
  directory_v1_df = pd.read_excel("directory_v1.xlsx")
  omstu_plus_df = pd.read_excel("OMSTU+.xlsx")
  omstu_plus_df_filter = omstu_plus_df.filter(["WoS", "Scopus", "full name"])
  directory_v1_df["names"] = directory_v1_df[["name in wos", "name in scopus"]].apply(tuple, axis=1)
  directory_v1_df["names"] = directory_v1_df["names"].apply(join_names)
  directory_v1_df.drop(["name in wos", "name in scopus"], inplace=True, axis=1)

  omstu_plus_df_filter["names_omstu+"] = omstu_plus_df_filter[["WoS", "Scopus"]].apply(tuple, axis=1)
  omstu_plus_df_filter["names_omstu+"] = omstu_plus_df_filter["names_omstu+"].apply(join_names)
  omstu_plus_df_filter.drop(["WoS", "Scopus"], inplace=True, axis=1)

  merge_data = pd.merge(left=directory_v1_df, right=omstu_plus_df_filter, left_on="Сотрудник", right_on="full name")
  merge_data.drop(["full name"], inplace=True, axis=1)
  merge_data.drop_duplicates(inplace=True)

  merge_data["names"] = merge_data[["names", "names_omstu+"]].apply(tuple, axis=1)
  merge_data["names"] = merge_data["names"].apply(just_join_names)
  merge_data.drop(["names_omstu+"], inplace=True, axis=1)

  names_list = merge_data["names"].to_list()
  names_list = filter_names(names_list)
  merge_data["names"] = names_list
  return merge_data


def get_osmtu_authors_wos(data_list: list) -> list:
  result_list: list = []
  for elem in data_list:
    for item in elem.split("; ["):
      if item.find("Omsk State Tech Univ") != -1:
        list_authors = item.split("]")[0]
        for author in list_authors.split("; "):
          if author[0] == "[":
            author = author[1:]
          if author[-1] != ";":
            author += ";"
            result_list.append(author.lower())
  return list(set(result_list))


def get_omstu_authors_scopus(data_list: list) -> list:
  result_list: list = []
  for elem in data_list:
    for item in elem.split("; "):
      if item.find("Omsk State Technical University") != -1:
        item_split = item.split(", ")
        author = item_split[0] + " " + item_split[1]
        author += ";"
        result_list.append(author.lower())
  return list(set(result_list))


def get_full_list(data_list: list) -> str:
  result_string: str = ""
  for i in data_list:
    if not isinstance(i, float):
      i = i.replace("  ", " ")
      result_string += i
  return result_string

In [14]:
wos_not_in_all_data: list = []
scopus_not_in_all_data: list = []
# ЕСЛИ ЕСТЬ ГОТОВЫЙ ФАЙЛ С ИМЕНАМИ ТО ФУНКЦИЮ get_from_ready_excel() вызывать не надо
# all_data: object = get_from_ready_excel()
all_data = pd.read_excel("result.xlsx")
scopus_df = pd.read_excel("scopus2020.xlsx")
wos_df = pd.read_excel("wos2020.xls")
wos_address_list = wos_df["Addresses"].to_list()
scopus_addres_list = scopus_df["Authors with affiliations"]
wos_authors_list = get_osmtu_authors_wos(wos_address_list)
scopus_authors_list = get_omstu_authors_scopus(scopus_addres_list)

all_data_names = all_data["names"].to_list()
all_data_names = get_full_list(all_data_names)

for i in wos_authors_list:
  if all_data_names.find(i[0:-1]) == -1:
    wos_not_in_all_data.append(i)
for i in scopus_authors_list:
  if all_data_names.find(i[0:-1]) == -1:
    scopus_not_in_all_data.append(i)

wos_df = pd.DataFrame({"WoS": wos_not_in_all_data})
scopus_df = pd.DataFrame({"Scopus": scopus_not_in_all_data})
wos_df.to_excel("wos_error.xlsx", index=False)
scopus_df.to_excel("scopus_error.xlsx", index=False)